In [ ]:
import os
import time
import json
import csv
import winsound

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

In [ ]:
def melody()-> None:
  '''Функция вызывает звуковые сигналы,
  предупреждающие о капче
  '''
    winsound.PlaySound("SystemExit", winsound.SND_ALIAS)
    winsound.PlaySound("SystemExit", winsound.SND_ALIAS)
    winsound.PlaySound('SystemHand', winsound.SND_ALIAS)
    winsound.PlaySound("SystemExit", winsound.SND_ALIAS)
    winsound.PlaySound("SystemExit", winsound.SND_ALIAS)

In [6]:
def anticaptcha_hands(bad_url: str)-> None:
  '''Функция открывает веб-страницу с капчей,
  дает время на ее прохождение вручную
  и передает кукис в вебдрайвер, чтобы
  капча не возникла на следующей странице
  '''
    driver.get(bad_url)
    time.sleep(10)
    se.headers.update(headers)
    for cookie in driver.get_cookies():
        c = {cookie['name']: cookie['value']}
        se.cookies.update(c)

Создаем пустой датафрейм с необходимыми столбцами

In [ ]:
link = "link"
title = "title"
addr = "addr"
price = "price"
subway = "subway"
subway_dist = "subway_dist"
highway = "highway"
highway_dist = "highway_dist"
flat_info = "flat_info"
obshchaya_info = "obshchaya_info"
house_info = "house_info"
opisanie = "opisanie"

with open(f"data10.csv", "w", encoding="utf-8") as file:
    writer = csv.writer(file, delimiter = '$', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(
    (
    link,
    title,
    addr,
    price,
    subway,
    subway_dist,
    highway,
    highway_dist,
    flat_info,
    obshchaya_info,
    house_info,
    opisanie
    )
)

Формируем список из объявлений, проходясь по всем страницам для каждого района Петербурга

In [ ]:
href = []
ob = 0 # счетчик объявлений

option = webdriver.ChromeOptions()
option.add_argument("--disable-notifications")
driver = webdriver.Chrome(os.getcwd()+"\chromedriver.exe", options=option) # держим driver открытым
time.sleep(5)

headers = {
        "Accept": "*/*",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.174 YaBrowser/22.1.5.810 Yowser/2.5 Safari/537.36"
    }
se = requests.session()


for distr in range(133, 150+1): # по каждому району, начало 133
    
    url = "https://spb.cian.ru/cat.php?deal_type=sale&district[0]="+str(distr)+"&engine_version=2&object_type[0]=1&offer_type=flat&p="
    if requests.get(url, headers=headers, cookies = se.cookies).text.find('Captcha')!=-1:
        anticaptcha_hands(url)

    it = 0   
    for item in range(1, 54+1): # по каждой странице
        it += 1 # счетчик страниц
        full_url = url+f"{item}"+"&region=2&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1"
        if requests.get(full_url, headers=headers, cookies = se.cookies).text.find('Captcha')!=-1:
            print("Captcha " * 10)
            time.sleep(15)
            anticaptcha_hands(full_url)

        req = requests.get(full_url, headers=headers, cookies = se.cookies) # защита от второй капчи подряд
        src = req.text
        if src.find('Captcha')!=-1:
            print("Captcha " * 10)
            time.sleep(15)
            anticaptcha_hands(full_url)
            req = requests.get(full_url, headers=headers, cookies = se.cookies)
            src = req.text

        soup = BeautifulSoup(src, "lxml")
        href_raw = soup.find_all(class_="_93444fe79c--link--eoxce")

        for item in href_raw: # для каждого объявления
            item_text = item.get("href")
            href.append(item_text)
            ob += 1

            print("Район №"+str(distr-132)+", Стр. №"+str(it)+", Объявление №"+str(ob))
driver.close()

Собираем информацию по каждому объявлению (=каждой ссылке) и помещаем в отдельный список

In [ ]:
s = 0 # счетчик объявлений
link = []
title = []
subway = []
subway_dist = []
highway = []
highway_dist = []
price = []
addr = []
flat_info = []
obshchaya_info = []
house_info = []
opisanie = []

option = webdriver.ChromeOptions()
option.add_argument("--disable-notifications")
driver = webdriver.Chrome(os.getcwd()+"\chromedriver.exe", options=option) # держим driver открытым
headers = {
        "Accept": "*/*",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.119 YaBrowser/22.3.0.2430 Yowser/2.5 Safari/537.36"
}
se = requests.session()


for i in range(0, len(href)):
    url = href[i]

    if requests.get(url, headers=headers, cookies=se.cookies).text.find('Captcha')!=-1:
        print("Captcha " * 10)
        melody()
        time.sleep(15)
        anticaptcha_hands(url)
    req = requests.get(url, headers=headers, cookies=se.cookies)
    src = req.text

    if src.find('Captcha')!=-1:
        print("Captcha " * 10)
        melody()
        time.sleep(15)
        anticaptcha_hands(url)
        req = requests.get(url, headers=headers, cookies=se.cookies)
        src = req.text
    soup = BeautifulSoup(src, "lxml")


    link.append(href[i])
    

    try:
        title_raw = soup.find_all(class_="a10a3f92e9--title--UEAG3")
        title_raw0 = title_raw[0]
    except IndexError:
        title.append("")
    else:
        if len(title_raw) > 1:
            li=[]
            for n in range(0, len(title_raw)):
                ww = title_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            title.append(string)
        else:
            title.append(title_raw[0].text.strip())
    
    try:
        addr_raw = soup.find_all(class_="a10a3f92e9--address--F06X3")
        addr_raw0 = addr_raw[0]
    except IndexError:
        addr.append("")
    else:
        if len(addr_raw) > 1:
            li=[]
            for n in range(0, len(addr_raw)):
                ww = addr_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            addr.append(string)
        else:
            addr.append(addr_raw[0].text.strip())


    try:
        flat_info_raw = soup.find_all(class_="a10a3f92e9--info-block--kXrDj")
        flat_info_raw0 = flat_info_raw[0]
    except IndexError:
        flat_info.append("")
    else:
        if len(flat_info_raw) > 1:
            li=[]
            for n in range(0, len(flat_info_raw)):
                ww = flat_info_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            flat_info.append(string)
        else:
            flat_info.append(flat_info_raw[0].text.strip())


    try:
        obshchaya_info_raw = soup.find_all(class_="a10a3f92e9--container--pZhdL")
        obshchaya_info_raw0 = obshchaya_info_raw[0]
    except IndexError:
        obshchaya_info.append("")
    else:
        if len(obshchaya_info_raw) > 1:
            li=[]
            for n in range(0, len(obshchaya_info_raw)):
                ww = obshchaya_info_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            obshchaya_info.append(string)
        else:
            obshchaya_info.append(obshchaya_info_raw[0].text.strip())


    try:
        house_info_raw = soup.find_all(class_="a10a3f92e9--content--ibqBi")
        house_info_raw0 = house_info_raw[0]
    except IndexError:
        house_info.append("")
    else:
        if len(house_info_raw) > 1:
            li=[]
            for n in range(0, len(house_info_raw)):
                ww = house_info_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            house_info.append(string)
        else:
            house_info.append(house_info_raw[0].text.strip())


    try:
        subway_raw = soup.find_all(class_="a10a3f92e9--underground_link--Sxo7K")
        subway_raw0 = subway_raw[0]
    except:
        subway.append("")
    else:
        if len(subway_raw) > 1:
            li=[]
            for n in range(0, len(subway_raw)):
                ww = subway_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            subway.append(string)
        else:
            subway.append(subway_raw[0].text.strip())


    try:
        subway_dist_raw = soup.find_all(class_="a10a3f92e9--underground_time--iOoHy")
        subway_dist_raw0 = subway_dist_raw[0]
    except IndexError:
        subway_dist.append("")
    else:
        if len(subway_dist_raw) > 1:
            li=[]
            for n in range(0, len(subway_dist_raw)):
                ww = subway_dist_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            subway_dist.append(string)
        else:
            subway_dist.append(subway_dist_raw[0].text.strip())


    try:
        highway_raw = soup.find_all(class_="a10a3f92e9--link--ulbh5 a10a3f92e9--highway_link--NOXPJ")
        highway_raw0 = highway_raw[0]
    except:
        highway.append("")
    else:
        if len(highway_raw) > 1:
            li=[]
            for n in range(0, len(highway_raw)):
                ww = highway_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            highway.append(string)
        else:
            highway.append(highway_raw[0].text.strip())


    try:
        highway_dist_raw = soup.find_all(class_="a10a3f92e9--highway_distance--wUNBn")
        highway_dist_raw0 = highway_dist_raw[0]
    except IndexError:
        highway_dist.append("")
    else:
        if len(highway_dist_raw) > 1:
            li=[]
            for n in range(0, len(highway_dist_raw)):
                ww = highway_dist_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            highway_dist.append(string)
        else:
            highway_dist.append(highway_dist_raw[0].text.strip())


    try:
        price_raw = soup.find_all(class_="a10a3f92e9--price_value--lqIK0")
        price_raw0 = price_raw[0]
    except IndexError:
        price.append("")
    else:
        if len(price_raw) > 1:
            li=[]
            for n in range(0, len(price_raw)):
                ww = price_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li])
            price.append(string)
        else:
            price.append(price_raw[0].text.strip())


    try:
        opisanie_raw = soup.find_all(class_="a10a3f92e9--description-text--YNzWU")
        opisanie_raw0 = opisanie_raw[0]
    except IndexError:
        opisanie.append("")
    else:
        if len(opisanie_raw) > 1:
            li=[]
            for n in range(0, len(opisanie_raw)):
                ww = opisanie_raw[n].text.strip()
                li.append(ww)
            string = ';'.join([str(item) for item in li]).replace("\n", "")
            opisanie.append(string)
        else:
            opisanie.append(opisanie_raw[0].text.strip().replace("\n", ""))

    s +=1
    print("Объявление №"+str(s)+", т.е."+str(s-1)+", осталось "+str(len(href)-s))

driver.close()

Заполняем датафрейм элементами из списков

In [ ]:
for i in range(0, len(addr)):
    with open(f"data10.csv", "a", encoding="utf-8") as file:
        writer = csv.writer(file, delimiter = '$', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(
        (
        link[i],
        title[i],
        addr[i],
        price[i],
        subway[i],
        subway_dist[i],
        highway[i],
        highway_dist[i],
        flat_info[i],
        obshchaya_info[i],
        house_info[i],
        opisanie[i]
        )
    )